In [1]:
!pip uninstall jupyternotify -y
!pip install git+https://github.com/cphyc/jupyter-notify.git
%reload_ext jupyternotify

Uninstalling jupyternotify-0.1.15:
  Successfully uninstalled jupyternotify-0.1.15
  Cloning https://github.com/cphyc/jupyter-notify.git to /tmp/pip-req-build-k_999jma
  Running command git clone -q https://github.com/cphyc/jupyter-notify.git /tmp/pip-req-build-k_999jma
  Stored in directory: /tmp/pip-ephem-wheel-cache-0lsipjmg/wheels/06/14/b3/01f4b7bbd1dc28a3e33d5163d145316c6714d9db6dd327947a
Successfully built jupyternotify


<IPython.core.display.Javascript object>

In [2]:
# load credentials from environment variables
%load_ext dotenv
%dotenv

# util
import numpy as np

# date & time
from datetime import timezone, date, datetime
from dateutil.relativedelta import relativedelta as rdelta
from dateutil.rrule import rrule, MONTHLY

# output processing and plotting
import io
import tarfile
import rasterio
from rasterio.plot import show
from matplotlib import pyplot

# Oauth
from oauthlib.oauth2 import BackendApplicationClient
from requests_oauthlib import OAuth2Session

## Get authorization token

In [4]:
# Your client credentials
client_id = %env SH_CLIENT_ID
client_secret = %env SH_CLIENT_SECRET

# Create a session
client = BackendApplicationClient(client_id=client_id)
oauth = OAuth2Session(client=client)

token = oauth.fetch_token(token_url='https://services.sentinel-hub.com/oauth/token',
                          client_id=client_id, client_secret=client_secret)

resp = oauth.get("https://services.sentinel-hub.com/oauth/tokeninfo")

## Configure request (evalscript)

Enter start and end date, input bands, indices. The resulting files will have two time intervals per month, being split at `day_of_new_interval`.

In [5]:
startdate = date(2018,7,5) # Y,M,D
enddate = date(2018,9,16)  # Y,M,D

input_bands = ["B04","B08", "B03"]
indices = ['NDVI']

day_of_new_interval = 16 # leave this unchanged in most of the cases

### Calculate parameters

In [6]:
starttime = datetime(*startdate.timetuple()[:6])
endtime = datetime(*enddate.timetuple()[:6])

d=day_of_new_interval
full_month_dates = list(rrule(MONTHLY, dtstart=startdate, until=enddate, bymonthday=[1,d-1,d,31]))
all_dates = [starttime] + full_month_dates + [endtime] # bounds of all intervals

starts = all_dates[0::2]
starts = [int(d.timestamp()) for d in starts] # timestamps for arithmetic
ends   = [d+rdelta(hour=23, minute=59, second=59) for d in all_dates[1::2]]
ends   = [int(d.timestamp()) for d in ends]   # timestamps for arithmetic
averages = list(np.mean(list(zip(starts,ends)), axis=1))
averages = [datetime.utcfromtimestamp(a) for a in averages]
averages = [dt.isoformat() for dt in averages]

In [7]:
masks = ["SCL", "dataMask"] # SCL ... Scene Classification Layer

output_bands = input_bands + indices
output_array =  ','.join([f"{{id: '{ob}', bands: {len(averages)}, "+
                          f"sampleType: SampleType.UINT16}}" for ob in output_bands])
int_bands = '{' + ','.join([f'{ib}: []' for ib in input_bands]) + '}'
results_object = '{' + ','.join([f'{ob}: []' for ob in output_bands]) + '}'
debug_results = '{' + ','.join([f"{output_bands[i]}: [{i+1}]" for i in range(len(output_bands))]) + '}'
responses = [{"identifier": ob,"format": {"type": "image/tiff"}} for ob in output_bands]

### Evalscript & Payload

In [8]:
#double curly brackets render as single curly brackets in f-strings
evalscript = f"""
    //VERSION=3
    
    var debug = []
    
    function setup(ds) {{
      return {{
        input: [{{
          bands: {str(input_bands + masks)}, 
          units: "DN"
        }}],
        output: [        
          {output_array}
        ],
        mosaicking: Mosaicking.ORBIT       
      }}
    }}

    function validate (sample) {{
      if (sample.dataMask!=1) return false;

      var scl = Math.round(sample.SCL);

      if (scl === 3) {{ // SC_CLOUD_SHADOW
        return false;
      }} else if (scl === 9) {{ // SC_CLOUD_HIGH_PROBA
        return false; 
      }} else if (scl === 8) {{ // SC_CLOUD_MEDIUM_PROBA
        return false;
      }} else if (scl === 7) {{ // SC_CLOUD_LOW_PROBA
        //return false;
      }} else if (scl === 10) {{ // SC_THIN_CIRRUS
        return false;
      }} else if (scl === 11) {{ // SC_SNOW_ICE
        return false;
      }} else if (scl === 1) {{ // SC_SATURATED_DEFECTIVE
        return false;
      }} else if (scl === 2) {{ // SC_DARK_FEATURE_SHADOW
        //return false;
      }}
      return true;
    }}

    function calculateIndex(a,b)
    {{
      if ((a+b)==0) return 0;
      var val = (a-b)/(a+b);
      if (val<0) val = 0;
      //TODO - we might need to return false instead of 0; depends on the output format - a value needs to be designated as "null"
      return val;
    }}

    function interpolatedValue(arr)
    {{
      //here we define the function on how to define the proper value - e.g. linear interpolation; we will use average 
      if (arr.length==0) return 0;
      if (arr.length==1) return arr[0];
      var sum = 0;
      for (i=0;i<arr.length;i++)
        {{sum+=arr[i];}}
      return Math.round(sum/arr.length);
    }}
    
    var results = {results_object}
    
    // We split each month into two halves. This will make it easier to append months to data cube later
    var day_of_new_interval = {day_of_new_interval}
    var endtime = new Date({endtime.timestamp()*1000}) // UNIX epoch in ms

    function evaluatePixel(samples, scenes, inputMetadata, customData, outputMetadata) {{

      //Debug part returning "something" if there are no  valid samples (no observations)
      if (!samples.length)
      return {debug_results}
      
      var is_in_last_half_of_month = endtime.getDate() >= day_of_new_interval
      var interval_number = 0;
      var int_bands = {int_bands}

      for (var i = 0; i < samples.length; i++) {{
        
        //TODO order should be reversed when we go leastRecent
        
        // if scene is outside of current half of month, fill result array and change half of month
        // algorithm starts with least recent observation
        if (( !is_in_last_half_of_month && scenes[i].date.getDate() >= day_of_new_interval) ||
            (  is_in_last_half_of_month && scenes[i].date.getDate() <  day_of_new_interval))
        {{
          fillResultArray(interval_number, int_bands)

          //reset values
          for (var int_b in int_bands) {{
            int_bands[int_b] = []
          }}

          is_in_last_half_of_month = !is_in_last_half_of_month;
          interval_number++;
        }}

        if (validate(samples[i]))
        {{
          // push input samples into their respective arrays
          for (var int_b in int_bands) {{
            int_bands[int_b].push(samples[i][int_b])
          }}
        }}

      }}

      //execute this for the last interval 
      fillResultArray(interval_number, int_bands);

      return results
    }}

    function fillResultArray(interval_number, int_bands)
    {{
      for (var b in int_bands) {{
        if(int_bands[b].length==0) results[b][interval_number] = 0
        else results[b][interval_number] = interpolatedValue(int_bands[b])
      }}
      
      for (var ix of {indices}) {{
        //TODO: fix for other indices
        results[ix][interval_number] = 65535*calculateIndex(results['B08'][interval_number],results['B04'][interval_number])
      }}
    }}
    
    function updateOutputMetadata(scenes, inputMetadata, outputMetadata) {{
      outputMetadata.userData = {{
        "date_created": Date(),
        "metadata": scenes.map(s => {{
          s.date = s.date.toString()
          return s
        }}),
        "time" : {averages},
        "debug": debug
      }}
    }}
"""

In [9]:
payload = {
  "processRequest": {
    "input": {
      "bounds": {
        "properties": {
          "crs": "http://www.opengis.net/def/crs/OGC/1.3/CRS84"
        },
        "bbox": [16.446445736463346, 47.680841561177864, 16.49776618971013, 47.72587417451863]
      },
      "data": [
        {
          "location": "AWS:eu-central-1",
          "type": "S2L2A",
          "dataFilter": {
            "timeRange": {
              "from": starttime.isoformat() + 'Z',
              "to": endtime.isoformat() + 'Z'
            },
            "mosaickingOrder": "mostRecent",
            "maxCloudCoverage": 100,
            "previewMode": "DETAIL"
          }
        }
      ]
    },
    "output": {
#       "width": 512,
#       "height": 512,
      "responses": [*responses#,
#         {
#           "identifier": "userdata",
#           "format": {
#             "type": "application/json"
#           }
#         }
      ]
    },
    "evalscript": evalscript
  },
  "tilingGridId": 0,
  "bucketName": "eox-masterdatacube",
  "resolution": 60.0,
  "description": "Test Loipersbach"
}

headers = {
  #'Accept': 'application/tar'
}

In [10]:
headers

{}

## Send request

In [11]:
def generate_url(request_id="", action=""):
    url = 'https://services.sentinel-hub.com/batch/v1/process/'
    if request_id:
        url += f'{request_id}/'
        if action:
            url += f'{action}'
    return url

In [12]:
%%time
response = oauth.request("POST", generate_url(), headers=headers, json = payload)

CPU times: user 3.73 ms, sys: 0 ns, total: 3.73 ms
Wall time: 1.03 s


In [13]:
response.status_code

201

In [14]:
response.json()

{'id': '6f260844-e480-4976-981a-4087c7204bef',
 'processRequest': {'input': {'bounds': {'bbox': [16.446445736463346,
     47.680841561177864,
     16.49776618971013,
     47.72587417451863],
    'geometry': None,
    'properties': {'crs': 'http://www.opengis.net/def/crs/OGC/1.3/CRS84'}},
   'data': [{'location': 'AWS:eu-central-1',
     'dataFilter': {'timeRange': {'from': '2018-07-05T00:00:00Z',
       'to': '2018-09-16T00:00:00Z'},
      'mosaickingOrder': 'mostRecent',
      'maxCloudCoverage': 100.0},
     'processing': None,
     'tileList': None,
     'id': None,
     'type': 'S2L2A'}]},
  'output': {'width': None,
   'height': None,
   'resx': None,
   'resy': None,
   'responses': [{'identifier': 'B04', 'format': {'type': 'image/tiff'}},
    {'identifier': 'B08', 'format': {'type': 'image/tiff'}},
    {'identifier': 'B03', 'format': {'type': 'image/tiff'}},
    {'identifier': 'NDVI', 'format': {'type': 'image/tiff'}}]},
  'dataProduct': None,
  'evalscript': '\n    //VERSION=3\

In [14]:
analysis_response = oauth.request("POST", generate_url(response.json()["id"],'analyse'))
analysis_response

<Response [204]>

In [15]:
oauth.request("POST", generate_url(response.json()["id"], 'start'))

<Response [204]>

In [18]:
%%notify -o
%%time
import time

response_status = ""
while response_status not in ['DONE', 'FAILED']:
    get_response = oauth.request("GET", generate_url(response.json()["id"]))
    response_status = get_response.json()['status']
    time.sleep(1)

response_status

CPU times: user 3.02 ms, sys: 409 µs, total: 3.43 ms
Wall time: 1.01 s


'DONE'

<IPython.core.display.Javascript object>

In [17]:
get_response.json()

{'id': '31f61ff9-e0af-4bfc-af2b-cd8d73fa719c',
 'processRequest': {'input': {'bounds': {'bbox': [14.318395625014208,
     48.359153946428485,
     14.506361110382839,
     48.543395524629055],
    'geometry': None,
    'properties': {'crs': 'http://www.opengis.net/def/crs/OGC/1.3/CRS84'}},
   'data': [{'location': None,
     'dataFilter': {'timeRange': {'from': '2018-07-01T00:00:00Z',
       'to': '2018-07-15T00:00:00Z'},
      'mosaickingOrder': 'mostRecent',
      'maxCloudCoverage': 100.0},
     'processing': None,
     'tileList': None,
     'id': None,
     'type': 'S2L2A'}]},
  'output': {'width': None,
   'height': None,
   'resx': None,
   'resy': None,
   'responses': [{'identifier': 'B04', 'format': {'type': 'image/tiff'}},
    {'identifier': 'B08', 'format': {'type': 'image/tiff'}},
    {'identifier': 'B03', 'format': {'type': 'image/tiff'}},
    {'identifier': 'NDVI', 'format': {'type': 'image/tiff'}}]},
  'dataProduct': None,
  'evalscript': '\n    //VERSION=3\n    \n    v